In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data


train_dir = "E:\\home\\workspace\\datasets\\mnist"
mnist = input_data.read_data_sets(train_dir=train_dir,one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:\home\workspace\datasets\mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:\home\workspace\datasets\mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting E:\home\workspace\datasets\mnist\t10k-images-idx3-ubyte.gz
Extracting E:\home\workspace\datasets\mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
x = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32,(None,10))

x.shape[0]

Dimension(None)

In [ ]:
def input_module(inputs,filters=64):
    outputs = tf.reshape(inputs,[-1,28,28,1])
    outputs = tf.layers.conv2d(outputs,filters=filters,kernel_size=3,padding='same')
    outputs = tf.layers.batch_normalization(outputs)
    outputs = tf.nn.relu(outputs)
    
    return outputs
    

In [ ]:
def residual_module(inputs,filters=64):
    outputs = tf.layers.conv2d(inputs,filters=filters,kernel_size=3,padding='same')
    outputs = tf.layers.batch_normalization(outputs)
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs,filters=filters,kernel_size=3,padding='same')
    outputs = tf.layers.batch_normalization(outputs)
    
    outputs += inputs
    outputs = tf.nn.relu(outputs)  
    
    return outputs

In [ ]:
def output_module(inputs,fc_units=256):
    outputs = tf.layers.conv2d(inputs,filters=1,kernel_size=1,padding='same')
    outputs = tf.layers.batch_normalization(outputs)
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.flatten(outputs)
    
    outputs = tf.layers.dense(outputs,fc_units,activation=tf.nn.relu)
    
    outputs = tf.layers.dense(outputs,10,activation=tf.nn.softmax)
    
    return outputs
    
    

In [ ]:
def inference(inputs,filters=16,fc_units=128,num_res_module=4):
    inputs = input_module(inputs,filters)
    
    mid_values = inputs
    for i_module in range(num_res_module):
        mid_values = residual_module(mid_values,filters)
        
    outputs = output_module(mid_values,fc_units)
    
    return outputs

In [ ]:
y_pred = inference(x)


In [ ]:
y_pred

In [ ]:
var = tf.trainable_variables()
regularizer = tf.contrib.layers.l2_regularizer(scale=.0001)
l2_var = tf.contrib.layers.apply_regularization(regularizer,var)

loss = tf.reduce_mean(tf.reduce_sum(-y*tf.log(y_pred),reduction_indices=[1])) + l2_var


optimizer = tf.train.AdamOptimizer().minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_pred,axis=1),tf.argmax(y,axis=1)),tf.float32))

In [ ]:
model_path = "./Model_save/model_resnet.ckpt"
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    try:
        saver.restore(sess,save_path=model_path)
        print("last model and params restore path:",model_path)
     
    except:
        print("not found restore path")
        
    finally:
        for it in range(1001):
            xs,ys = mnist.train.next_batch(128)
            sess.run(optimizer,feed_dict={x:xs,y:ys})

            if it % 100 == 0:
                test_xs,test_ys = mnist.test.next_batch(1000)
                train_xs, train_ys = mnist.train.next_batch(1000)
                
                v_loss, v_accuracy = sess.run([loss,accuracy],{x:test_xs,y:test_ys})
                print("> Iter num: %d, Test loss is %.2f, accuracy is %.4f"%(it+1,v_loss,v_accuracy))
                
                v_loss, v_accuracy = sess.run([loss,accuracy],{x:train_xs,y:train_ys})
                print("> Iter num: %d, Trian loss is %.2f, accuracy is %.4f"%(it+1,v_loss,v_accuracy))

        saver.save(sess=sess,save_path=model_path)
        print("latest model and params save path:",model_path)